# Graph SLAM (Simultaneous Localization And Mapping)


SLAM is a neat techique since it allows both localization of a robot and mapping the world at the same time and it can be implemented in an incremental way.

The basic idea of SLAM is that we have a sequence of measurements or movements, we can create two matrices $\Omega$ and $\xi$. For each measured sequence $x_i - x_j = c_k$, we update the $\Omega$ and $\xi$ in the following way:
$$\Omega_{i,i} \ +=1; \Omega_{i,j} \  -=1; \xi_{i} \  += \ c_k$$
$$\Omega_{j,i} \ -= \ 1; \Omega_{j,j} \ += \ 1; \xi_{j} \ -= \ c_k$$
After the update, the position of each x can be computed by $$x=\Omega ^{-1} \xi$$
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;A more detailed illustration of this process can be found [here](https://www.youtube.com/watch?time_continue=3&v=V41eTlGU0gw)

Here, I would like to talk about why this works. The key is that we have a series of locations (including the robot positions and landmark positions) $x$, assume that we want to find a good approximation to the true locations. After each movement, the probability of the target at locations $x$ can be expressed as gaussian equations. 
$$P(x_t | x_{t-1}, x_{t_2}, ..., c_{k}, c_{k-1},...)=const* exp(-\frac {1}{2}[(x_t - x_{t-1}-c_{k})^2+(x_{t-1}-x_{t-2}-c_{k-1})^2+...])$$
In order to maximize the probability, we need to minimize 
$$f=(x_t - x_{t-1}-c_{k})^2+(x_{t-1}-x_{t-2}-c_{k-1})^2+...=\sum (x_i - x_j -c_k)^2 $$
To minimize this, we need to have $$\frac {df}{dx_i} =0$$ for any $i$. 
For any $x_i$ with a plus sign in the equation, we get $$2(x_i-x_j-c_k)$$, so we have the first line of the update rule, for any $x_j$ with a minus sign, we get $$-2(x_i-x_j-c_k)=2(-x_i+x_j+c_k)$$, so we get the second line of the update rule. (Note we removed the 2 in the because it is the same for every line) <br>
Since different measurement may have different accuracy, one way is to extend the $f$ function by
$$f=\sum w_k(x_i - x_j -c_k)^2$$ So that measurement with more confidence can be assigned with higher weight $w_k$

### Example:
we have 3 positions and 2 landmarks, we denote positions as 0, 1, 2 and landmarks as 3 and 4. We get the following measurements

In [1]:
meas = [
    [1, 0, 3],  # means x1 - x0 =3
    [2, 1, 6],
    [3, 0, 4],
    [3, 2, -5],
    [4, 1, 4],
    [4, 2, -2]
]

# initial value
init = [[0, 0]]  # x0 = 0

# create matrix based on the above information
import numpy as np

Omega = np.matrix(np.zeros((5, 5), dtype=np.float))
Xi = np.matrix(np.zeros((5, 1), dtype=np.float))
# Update the matrix using the above information
for key in init:
    i = key[0]
    c = key[1]
    Omega[i, i] += 1
    Xi[i] += c
for key in meas:
    i = key[0]
    j = key[1]
    c = key[2]
    Omega[i, i] += 1
    Omega[i, j] += -1
    Xi[i] += c
    Omega[j, i] += -1
    Omega[j, j] += 1
    Xi[j] += -c

## print out the matrices
print("----------Omega--------------\n", Omega)
print("-----------Xi----------------\n", Xi)

----------Omega--------------
 [[ 3. -1.  0. -1.  0.]
 [-1.  3. -1.  0. -1.]
 [ 0. -1.  3. -1. -1.]
 [-1.  0. -1.  2.  0.]
 [ 0. -1. -1.  0.  2.]]
-----------Xi----------------
 [[ -7.]
 [ -7.]
 [ 13.]
 [ -1.]
 [  2.]]


In [2]:
from numpy.linalg import inv

## now let's print out the coordinate of the landmarks and positions
x = np.dot(inv(Omega), Xi)
print("-----------coordinates----------\n", x)

-----------coordinates----------
 [[ 0.]
 [ 3.]
 [ 9.]
 [ 4.]
 [ 7.]]


It turns out the the above result gives exactly the measurement in meas. <br>
now let's try to add some noise


In [3]:
np.random.seed(3965487)
for eachitem in meas:
    eachitem[2] += eachitem[2] * 0.05 * (np.random.random() - 0.5)
print("----------noisy measurements----------\n", meas)

----------noisy measurements----------
 [[1, 0, 3.0532128474193785], [2, 1, 5.924411460534505], [3, 0, 4.0401340282831475], [3, 2, -4.915820628202132], [4, 1, 3.955000197181814], [4, 2, -2.0312694345492623]]


In [4]:
## rerun the calculations
Omega = np.matrix(np.zeros((5, 5)))
Xi = np.matrix(np.zeros((5, 1)))
# Update the matrix using the above information
for key in init:
    i = key[0]
    c = key[1]
    Omega[i, i] += 1
    Xi[i] += c
for key in meas:
    i = key[0]
    j = key[1]
    c = key[2]
    Omega[i, i] += 1
    Omega[i, j] += -1
    Xi[i] += c
    Omega[j, i] += -1
    Omega[j, j] += 1
    Xi[j] += -c
x = np.dot(inv(Omega), Xi)
print("-----------coordinates----------\n", x)

-----------coordinates----------
 [[ 0.        ]
 [ 3.04167947]
 [ 8.97902141]
 [ 4.0516674 ]
 [ 6.97221582]]


The above result shows that we can still make a good estimation

## Online SLAM
sometime, we are only interested in the position of landmarks, and the last position of the moving object. We want to avoid the expansion of the matrices. The way to do it is that we use online SLAM. To make this happen, we simply need to find a way to remove a row and a column from the matrix, without modifying the the result of other position. For an equation $$\Omega x=\xi$$ it simply means that $$\sum \Omega_{i,j}x_j = \xi _i $$ for any $i$. Suppose we want to remove the first element $x_1$, we simply need to represent $x_i$ with other $x$s and substitue it into all other rows. So the update rule is $$ \Omega_{i,j}+=-\frac {\Omega_{i,1}}{\Omega_{1,1}} {\Omega_{1,j}}, \ \ \ \ \xi_i+=-\frac {\Omega_{i,1}}{\Omega_{1,1}} \xi_1 $$ for any $i \neq 1$. For any other elements $k$, we simply need to replace 1 with $k$

In [5]:
## Now let's use the above update rule and see if it works, because x0==0, let's remove x4
print("-----before remove x4---------")
print(Omega)
print(Xi)
for i in range(0, 4):
    factor = -(1.0 * Omega[i, 4] / Omega[4, 4])
    for j in range(5):
        Omega[i, j] += factor * Omega[4, j]
    Xi[i, 0] += factor * Xi[4, 0]

Omega = Omega[0:4, 0:4]
Xi = Xi[0:4, :]
print("-----after remove x4---------")
print(Omega)
print(Xi)
# print(Omega[0,0])
x = np.dot(inv(Omega), Xi)
print("-----------coordinates----------\n", x)

-----before remove x4---------
[[ 3. -1.  0. -1.  0.]
 [-1.  3. -1.  0. -1.]
 [ 0. -1.  3. -1. -1.]
 [-1.  0. -1.  2.  0.]
 [ 0. -1. -1.  0.  2.]]
[[ -7.09334688]
 [ -6.82619881]
 [ 12.87150152]
 [ -0.8756866 ]
 [  1.92373076]]
-----after remove x4---------
[[ 3.  -1.   0.  -1. ]
 [-1.   2.5 -1.5  0. ]
 [ 0.  -1.5  2.5 -1. ]
 [-1.   0.  -1.   2. ]]
[[ -7.09334688]
 [ -5.86433343]
 [ 13.8333669 ]
 [ -0.8756866 ]]
-----------coordinates----------
 [[ 0.        ]
 [ 3.04167947]
 [ 8.97902141]
 [ 4.0516674 ]]


The result shows that the x0 to x3 values are the same as the one calculated before. Using this technique, one is able to keep track of a robot's last position, without building a huge matrix since all the past positions are fogotten